# Tutorial on DER Hosting Capacity | <font color=red>Part 0</font>: Using DSS-Python

## 1. Introduction 

### Objectives 
The objectives of this tutorial are:
1. To familiarise with **advanced tools useful to run distribution network studies involving DERs**. You will be using the programming language Python and the advanced distribution network analysis tool [OpenDSS](https://www.epri.com/pages/sa/opendss) via the [DSS-Python](https://github.com/dss-extensions/DSS-Python) module. And, to guide you, all will be done using a notebook on [Jupyter Notebook](https://jupyter.org/).

2. To interact with the **DSS-Python module** and show how it replaces using OpenDSS via the COM interface. This module makes simulations and the handling of data/results much faster as everything will be native to Python.

<font color='red'>**<u>Note</u>:**</font> The use of dss-python implies you will be using (and, therefore, you will be limited to) the embedded models and control functions of OpenDSS for particular elements. For instance, you can use OpenDSS's Volt-Watt function for solar PV systems. But you will not be able to modify it to do simultaneous Volt-Watt and Volt-var. If you need to create specific functionalities in Python, then you will need to interact with OpenDSS via the COM interface (which can also be done using Python).


### Structure of this Notebook
The rest of this notebook is divided into two parts:

- **2. Tutorial.** Here you will learn, step by step, how to model a distribution network and analyse what-if scenarios by running power flows with OpenDSS and Python.
- **3. Simulation Workspace.** Here you can place all your code to run it at once.

## 2. Tutorial
### 2.1 Check both libraries are installed
#### Initialise OpenDSS object via COM interface

In [ ]:
import win32com.client
dss_via_com = win32com.client.Dispatch("OpenDSSEngine.DSS") # does NOT use early binding
print(dss_via_com) # check object type

#### Initialise OpenDSS object via dss library (already installed using the DSS-Python module)

In [ ]:
import dss
dss_via_python = dss.DSS
print(dss_via_python) # check object type

#### Import other libraries

In [ ]:
import time # to capture the time taken to run the simulations
import matplotlib.pyplot as plt # to plot the results
import numpy as np # to handle numerical data

### 2.2 Create simple network and run simulations
#### 2.2.1 Initialise data for the network

In [ ]:
text_commands = [
    'clear',
    'set DefaultBaseFrequency=50',
    'new circuit.test_lv_feeder bus1=slack basekv=0.4 pu=1.0 angle=0 frequency=50 phases=3',    
    'new line.slack-B1 phases=3 bus1=slack bus2=B1 r1=0.1 x1=0.1 r0=0.05 x0=0.05 length=1',
    'new line.B1-B2 phases=3 bus1=B1 bus2=B2 r1=0.1 x1=0.1 r0=0.05 x0=0.05 length=1',
    'new line.B2-B3 phases=3 bus1=B2 bus2=B3 r1=0.1 x1=0.1 r0=0.05 x0=0.05 length=1',
    'new loadshape.demand npts=24 interval=1.0 mult={1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 5.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 5.0, 7.0, 7.0, 5.0, 3.0, 1.0, 1.0,}',
    'new loadshape.solar  npts=24 interval=1.0 mult={0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 1.0, 0.8, 0.7, 0.5, 0.3, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,}',   
    'new load.house phases=1 bus1=B3.1 kv=0.23 kw=1 kvar=0 vmaxpu=1.5 vminpu=0.8 daily=demand',
    'new generator.pv_system phases=1 bus1=B3.2 kv=0.23 kw=5 pf=1 vmaxpu=1.5 vminpu=0.8 daily=solar',
    'reset',
    'set ControlMode=Time',
    'set Mode=Daily StepSize=1h Number=1 Time=(0,0)',
    'set VoltageBases=[0.4]',
    'calcv',    
]

#### 2.2.2 Create network using dss_via_com and run simulation

In [ ]:
start_com = time.time()   # to capture the starting time dss via com
# only run this once
dss_via_com.Start(0)
dss_via_com.AllowForms = True

In [ ]:
# Read the commands
for cmd in text_commands:
    dss_via_com.Text.Command = cmd
    
n_hours = 24 # Set the number of hours to run the simulation
data_com = {
    'PV System': {'element_name': 'generator.pv_system', 'Power (kW)': [], 'Voltage (V)': []},
    'House': {'element_name': 'load.house', 'Power (kW)': [], 'Voltage (V)': []}
}
for t in range(n_hours):
    dss_via_com.ActiveCircuit.Solution.Solve()
    
    for element in data_com.keys():
        dss_via_com.ActiveCircuit.SetActiveElement(data_com[element]['element_name'])
        data_com[element]['Power (kW)'].append(dss_via_com.ActiveCircuit.ActiveElement.Powers[0])
        data_com[element]['Voltage (V)'].append(dss_via_com.ActiveCircuit.ActiveElement.VoltagesMagAng[0])

end_com = time.time() # to capture the ending time dss via com

#### Plot results
This will plot the power and voltage of the PV System and House elements in the network using the COM interface.

In [ ]:
# Create a figure and subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
plt.suptitle('Using COM interface')

primary_keys = list(data_com.keys())
inner_keys = [key for key in data_com[primary_keys[0]].keys() if key != 'element_name']

# Iterate over rows (primary keys) and columns (inner keys)
for i, primary_key in enumerate(primary_keys):
    for j, inner_key in enumerate(inner_keys):
        ax = axes[i, j]  # Select subplot
        # Set column title for the first row only
        if i == 0: ax.set_title(inner_key, fontsize=12, fontweight='bold')
        # Set row label for the first column only
        if j == 0: ax.set_ylabel(primary_key, rotation=90, fontsize=12, fontweight='bold')
            

        # Example placeholder for empty data
        ax.plot(data_com[primary_key][inner_key])  # Replace with actual data if available
        ax.set_xticks([0, 6, 12, 18, 24])
        
plt.show()

#### 2.2.3 Repeat the same using dss_via_python
<font color='red'>**<u>Note 1</u>:**</font> If you copy the code to Python, remember to copy first the code from points 1.2 and 2.1.

In [ ]:
start_dss = time.time()  # to capture the starting time for dss via python
dss_via_python.Start(0)

# Read the commands
for cmd in text_commands:
    dss_via_python.Text.Command = cmd
    
n_hours = 24 # Set the number of hours to run the simulation
data_python = {
    'PV System': {'element_name': 'generator.pv_system', 'Power (kW)': [], 'Voltage (V)': []},
    'House': {'element_name': 'load.house', 'Power (kW)': [], 'Voltage (V)': []}
}
for t in range(n_hours):
    dss_via_python.ActiveCircuit.Solution.Solve()
    
    for element in data_python.keys():
        dss_via_python.ActiveCircuit.SetActiveElement(data_python[element]['element_name'])
        data_python[element]['Power (kW)'].append(dss_via_python.ActiveCircuit.ActiveElement.Powers[0])
        data_python[element]['Voltage (V)'].append(dss_via_python.ActiveCircuit.ActiveElement.VoltagesMagAng[0])

end_dss = time.time() # to capture the ending time dss via com

#### Plot results again, should match perfectly with the one above
This will plot the power and voltage of the PV System and House elements in the network using the DSS-Python module.

In [ ]:
# Create a figure and subplots
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
plt.suptitle('Using DSS-Python library')

primary_keys = list(data_python.keys())
inner_keys = [key for key in data_python[primary_keys[0]].keys() if key != 'element_name']

# Iterate over rows (primary keys) and columns (inner keys)
for i, primary_key in enumerate(primary_keys):
    for j, inner_key in enumerate(inner_keys):
        ax = axes[i, j]  # Select subplot
        # Set column title for the first row only
        if i == 0: ax.set_title(inner_key, fontsize=12, fontweight='bold')
        # Set row label for the first column only
        if j == 0: ax.set_ylabel(primary_key, rotation=90, fontsize=12, fontweight='bold')
            

        # Example placeholder for empty data
        ax.plot(data_python[primary_key][inner_key])  # Replace with actual data if available
        ax.set_xticks([0, 6, 12, 18, 24])
        
plt.show()

#### 2.2.4 Check numerical differences and simulation times

Difference between the results obtained using the COM interface and DSS-Python.

In [ ]:
print(np.array(data_com['PV System']['Voltage (V)']) - np.array(data_python['PV System']['Voltage (V)']))

In [ ]:
print(np.array(data_com['PV System']['Power (kW)']) - np.array(data_python['PV System']['Power (kW)']))

Time taken to run the simulations.

In [ ]:
print(end_com-start_com, 'seconds')

In [ ]:
print(end_dss-start_dss, 'seconds')

### 2.3 Key Remarks
Running OpenDSS via Python (using the **DSS-Python module**) is generally many times faster than running OpenDSS via the COM interface. Although the time difference above might look small, the gains are very evident when running large, complex simulations.

## 3. Simulation Workspace